# 네이버 카페 contents 크롤링
제목/ 닉네임/ 날짜 / 내용/ 댓글 데이터 한 페이지에서 가져오기

In [1]:
import requests
import pandas as pd

import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import urllib.request
from bs4 import BeautifulSoup

### 2015 네이버 키워드 '쌀'
: 쌀 -업체, -제공받, -지원받, -원고료, -무상, -협찬, -소정의, -사료, -고래밥, -길냥이, -고양이, -강아지
https://search.naver.com/search.naver?sm=tab_hty.top&where=article&query=%EC%8C%80&oquery=Tkf&tqi=hdzB%2Fdp0YihssA3p2MGssssss3s-479049&nso=so%3Ar%2Cp%3Afrom20150101to20151231

## 01월
https://search.naver.com/search.naver?where=article&query=%EC%8C%80%20-%EC%97%85%EC%B2%B4%2C%20-%EC%A0%9C%EA%B3%B5%EB%B0%9B%2C%20-%EC%A7%80%EC%9B%90%EB%B0%9B%2C%20-%EC%9B%90%EA%B3%A0%EB%A3%8C%2C%20-%EB%AC%B4%EC%83%81%2C%20-%ED%98%91%EC%B0%AC%2C%20-%EC%86%8C%EC%A0%95%EC%9D%98%2C%20-%EC%82%AC%EB%A3%8C%2C%20-%EA%B3%A0%EB%9E%98%EB%B0%A5%2C%20-%EA%B8%B8%EB%83%A5%EC%9D%B4%2C%20-%EA%B3%A0%EC%96%91%EC%9D%B4%2C%20-%EA%B0%95%EC%95%84%EC%A7%80&ie=utf8&st=rel&date_option=99&date_from=2015.01.01&date_to=2015.01.31&board=&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20150101to20150131&nso_open=1&t=0&mson=0&prdtype=0

In [37]:
#쌀
driver = webdriver.Chrome('./chromedriver')  
driver.get('https://search.naver.com/search.naver?where=article&query=%EC%8C%80%20-%EC%97%85%EC%B2%B4%2C%20-%EC%A0%9C%EA%B3%B5%EB%B0%9B%2C%20-%EC%A7%80%EC%9B%90%EB%B0%9B%2C%20-%EC%9B%90%EA%B3%A0%EB%A3%8C%2C%20-%EB%AC%B4%EC%83%81%2C%20-%ED%98%91%EC%B0%AC%2C%20-%EC%86%8C%EC%A0%95%EC%9D%98%2C%20-%EC%82%AC%EB%A3%8C%2C%20-%EA%B3%A0%EB%9E%98%EB%B0%A5%2C%20-%EA%B8%B8%EB%83%A5%EC%9D%B4%2C%20-%EA%B3%A0%EC%96%91%EC%9D%B4%2C%20-%EA%B0%95%EC%95%84%EC%A7%80&ie=utf8&st=rel&date_option=99&date_from=2015.01.01&date_to=2015.01.31&board=&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20150101to20150131&nso_open=1&t=0&mson=0&prdtype=0')
time.sleep(0.5)

In [38]:
# second 동안 스크롤다운 함수
import datetime

def doScrollDown(whileSeconds):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        if datetime.datetime.now() > end:
            break
        

In [6]:
doScrollDown(100)

In [39]:
# 크롤링할 주소 동적가져오기
article_list = driver.find_elements_by_css_selector('.api_txt_lines.total_tit')
len(article_list)

10

## 1. 카페 글 접속하기

In [53]:
#개별 패이지주소 들어감
article_1 = article_list[0]
article_1.click() # 첫번째 주소 입장

In [48]:
# 열린 창 확인
driver.window_handles 

['CDwindow-C06F7AB683FE07C9A737C319E6B3B790',
 'CDwindow-0C0381466C43EE275BDC01E0A61180A4']

In [56]:
# 드라이버 윈도우 설정
change_tab = driver.window_handles[-1]
driver.switch_to.window(change_tab)

## 2. 필요데이터 수집하기

In [57]:
# 프레임 변경
driver.switch_to.frame('cafe_main') #크롤링할 데이터 존재frame

In [58]:
# 페이지 준비
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [81]:
# 제목/ 닉네임/ 날짜 / 내용/ 댓글
title = soup.select('h3.title_text')[0].text.strip() #h3태그에 title_text클래스
nickname = soup.select('a.nickname')[0].text.strip() #a테그에 nickname클래스
date = soup.select('span.date')[0].text[:10] #시간제외 날짜까지만 인덱싱하여 가져옴
content = soup.select('div.ContentRenderer')[0].text #div태그에 ContentRenderer클래스

# 댓글 : 여러개 존재하므로 모두가져온다음 text만 뽑음 
comments_all = soup.select('span.text_comment')
comment_list =[]
for comment in comments_all:
    comment = comment.text.strip()
    comment_list.append(comment)
# 댓글 : 리스트-> 문자열로 저장
seperator = '||'
comments = seperator.join(comment_list)

# 제목/ 닉네임/ 날짜 / 내용/ 댓글 데이터-> 리스트로 저장
post = [title , nickname, date, content,comments]
post

['빨간쌀 필요하신분이요^^',
 '악기하는 여자',
 '2015.01.04',
 '저희 큰아버지께서 쌀도매를 하세요^^빨간쌀 이예요.저도 작녁부터 계속 먹고있는데요^^검정쌀처럼 찹쌀적미라 밥에 섞어 드시는건데요.항암효과.당뇨예방.노화방지~~빨간색성분은 몸에 좋다고들 하잖아요.저 아시는 지인분들께서 구매를 원하셔서 오늘 시골왔다가 내일 올라가는데요^^이번에 새로농사지은 햇찹쌀이예요♡2.5키로에 15000원이구요.시중에서는 보기힘든 귀한쌀이구요.백화점에서는1키로에 1만원에 판매되고 있는 비싼쌀이예요.필요하신분들~댓글주세요♡제가 차로 가지고 가는거라~~많이는 못가져가요^^;;\xa0\xa0\xa0\xa0',
 '네^^일반쌀 검정쌀 처럼 섞어드시는거구요♡\n구입 하시는분 많으심~~메타로 가져다 드리구요^^많이 없으심~제가 집으로가져다 드릴께요^^||01050221953으로 키로수랑 문자주세요♡||네^^2.5키로가  사진에 보이는 팩으로 한팩이예요♡||01050221953으로 문자로 키로수랑 알려주세요♡||01050221953으로 키로수 문자주세요♡||네^^01050221953으로 문자주세요♡||키로수가~2.5키로가 기본키로수15000원이예요♡\n01050221953으로 문자주세요♡']

## 3. 열린 창 닫기

In [49]:
driver.close()

In [50]:
driver.window_handles # 열린 창 확인

['CDwindow-C06F7AB683FE07C9A737C319E6B3B790']

In [52]:
# 다른 게시글 들어갈 준비 (必)
change_tab = driver.window_handles[-1]
driver.switch_to.window(change_tab)

## 4. 데이터 엑셀파일에 저장

In [97]:
# title , nickname, date, content,comments 데이터프레임으로 저장
df = pd.DataFrame(post)
df_T = df.transpose()
df_T

,0,1,2,3,4
0,빨간쌀 필요하신분이요^^,악기하는 여자,2015.01.04,저희 큰아버지께서 쌀도매를 하세요^^빨간쌀 이예요.저도 작녁부터 계속 먹고있는데요^...,네^^일반쌀 검정쌀 처럼 섞어드시는거구요♡\n구입 하시는분 많으심~~메타로 가져다 ...


In [95]:
writer = pd.ExcelWriter('naver_cafe_ssal_contents_2015.xlsx')
df.to_excel(writer,index =False)
writer.save()